In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive

##drive.mount("/content/gdrive")

path = '/content/gdrive/My Drive/Assignment Data/data/Asset_Price'
filelist=[]
for files in os.listdir(path):
    if os.path.splitext(files)[1] == '.csv':
        filelist.append(files)
symbol = [i[:-4] for i in filelist]
symbol.remove('SPY')

# Reading Adjusted Close Price of Each Investment 

df_p = pd.DataFrame(columns=symbol)
asset_price= []
for i in symbol:
    df_i = pd.read_csv(os.path.join('/content/gdrive/My Drive/Assignment Data/data/Asset_Price',i+'.csv'),index_col=0)
    df_p[i] = df_i['Adj Close']

#Reading SPY Data
df_i = pd.read_csv(os.path.join('/content/gdrive/My Drive/Assignment Data/data/Asset_Price','SPY.csv'),index_col=0)
df_m = df_i.loc[:,'Adj Close']
#calculate daily return 
df_r = (df_p/df_p.shift(1)-1).dropna()
df_mr = (df_m/df_m.shift(1)-1).dropna()
df_r.head()

#Reading Fama French Model Factors
FFM_df = pd.read_csv('/content/gdrive/My Drive/Assignment Data//data/FFM1.csv',index_col=0)
FFM_df = pd.DataFrame(FFM_df)
FFM_df.index = pd.to_datetime(FFM_df.index,format='%Y%m%d')
FFM_df.head()
#####Starting Regression to Calculate Alpha and fitting it#####
from sklearn.linear_model import LinearRegression

def regress(mu_term, sigma_term, date, security, french_fama):

fama_pos = french_fama.index.get_loc(date)
rf = french_fama.iloc[fama_pos - mu_term: fama_pos]['RF']
    factorMkt = french_fama.iloc[fama_pos - mu_term: fama_pos]['Mkt-RF']
    factorSMB = french_fama.iloc[fama_pos - mu_term: fama_pos]['SMB']
    factorHML = french_fama.iloc[fama_pos - mu_term: fama_pos]['HML']

    X = pd.concat([factorMkt, factorSMB, factorHML], axis=1)

    return_pos = security.index.get_loc(date)
    y = security.iloc[return_pos - mu_term: return_pos] - rf

    reg = LinearRegression().fit(X, y)
    vol = reg.predict(X) + rf
    mu_re= np.mean(vol)

    # regression for sigma
    rf = french_fama.iloc[fama_pos - sigma_term: fama_pos]['RF']
    factorMkt = french_fama.iloc[fama_pos - sigma_term: fama_pos]['Mkt-RF']
    factorSMB = french_fama.iloc[fama_pos - sigma_term: fama_pos]['SMB']
    factorHML = french_fama.iloc[fama_pos - sigma_term: fama_pos]['HML']
    X = pd.concat([factorMkt, factorSMB, factorHML], axis=1)
    y = security.iloc[return_pos - sigma_term: return_pos] – rf
reg = LinearRegression().fit(X, y)
    vol = reg.predict(X) + rf
    rm = factorMkt + rf
    ri = security.iloc[return_pos - sigma_term: return_pos]
    beta_i_m = rm.cov(ri) / rm.cov(rm)
    return mu_return, vol, beta_i_m
#regression before crisis：
date = weekday_before.iloc[0]['date'] # rebalance beginning
securities = df_r.columns
volatility_before= []
beta_before = []
sigma_before = pd.DataFrame(columns=securities)

for sec in securities:
    mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
    volatility_before.append(mu_re)
    sigma_before[sec]= mu_sig
    beta_before.append(beta_i)
sigma_before = sigma_table_before.cov()
#regression：
date = weekday_during.iloc[0]['date'] 
securities = df_r.columns
volatility_during = []
beta_during = []
sigma_during = pd.DataFrame(columns=securities)

for sec in securities:
    mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
    volatility_during.append(mu_re)
    sigma_during[sec]= mu_sig
    beta_during.append(beta_i)
sigma_during = sigma_during.cov()

#regression after：
date = weekday_after.iloc[0]['date'] 
securities = df_r.columns
volatility_after = []
beta_after = []
sigma_after = pd.DataFrame(columns=securities)

for sec in securities:
    mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
    volatility_after.append(mu_re)
    sigma_after[sec]= mu_sig
    beta_after.append(beta_i)
sigma_after = sigma_table_after.cov()
from cvxopt import matrix, solvers
def get_weights(vol,cov,wp,betam,betaT,lamb):
n=len(cov)
    P=matrix(np.array(2*lamb*cov))

    q=matrix(-1*(vol+2*lamb*np.array(wp)@cov))
    G=matrix(np.vstack((-1*np.eye(n),np.eye(n))))
    h=matrix(np.hstack((2*np.ones(n),2*np.ones(n))).T)
    A=matrix([list(betam),[1.0]*n])
    b = matrix([float(betaT),1.0],(2,1))
    
    sol=solvers.qp(P,q, G, h, A.T, b)

    return np.array(sol["x"]),blas.dot(matrix(np.array(vol)), sol["x"])
wp = [1/11]*11
optimal_w_before = get_weights(vols_before,sigma_before,wp,betas_before,betaT=1,lamb=0.8)[0]
optimal_r_before=get_weights(vols_before,sigma_before,wp,betas_before,betaT=1,lamb=0.8)[1]

optimal_w_during = get_weights(vols_during,sigma_during,wp,betas_during,betaT=1,lamb=0.8)[0]
optimal_r_during=get_weights(vols_during,sigma_during,wp,betas_during,betaT=1,lamb=0.8)[1]

optimal_w_after = get_weights(vols_after,sigma_after,wp,betas_after,betaT=1,lamb=0.8)[0]
optimal_r_after=get_weights(vols_after,sigma_after,wp,betas_after,betaT=1,lamb=0.8)[1]
beta1=
beta2= 
beta3=

### Weights for Regression Before Crisis.  Change Values 9 Times for beta and return covariance lookback period
bop_w_40_60_beta1=[]
bop_w_40_60_beta1.append(optimal_w_before )
#op_r=[]
#op_r.append(optimal_r )
for i in range(len(weekday_before)):
    date = weekday_before.iloc[i][0] # rebalance according to the first trading date
    securities = df_r.columns
    vols = []
    betas = []
    sigma_table = pd.DataFrame(columns=securities)

    for sec in securities:
        mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
        vols.append(mu_re)
        sigma_table[sec]= mu_sig
        betas.append(beta_i)
    sigma = sigma_table.cov()
    
#optimization
    op=get_weights(vols,sigma,bop_w_40_60_beta1[-1].flatten().tolist() ,betas,betaT=beta1,lamb=0.8)
    optimal_w =op [0]
    bop_w_40_60_beta1.append(optimal_w)

    ### Weights for Regression Before Crisis.  Change Values 9 Times for beta and return covariance lookback period
dop_w_40_60_beta1=[]
dop_w_40_60_beta1.append(optimal_w_during )
for i in range(len(weekday_during)):
    date = weekday_during.iloc[i][0] 
    securities = df_r.columns
    vols = []
    betas = []
    sigma_table = pd.DataFrame(columns=securities)

    for sec in securities:
        mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
        vols.append(mu_re)
        sigma_table[sec]= mu_sig
        betas.append(beta_i)
    sigma = sigma_table.cov()
    
#optimization
    op=get_weights(vols,sigma,dop_w_40_60_beta1[-1].flatten().tolist() ,betas,betaT=beta1,lamb=0.8)
    optimal_w =op [0]
    dop_w_40_60_beta1.append(optimal_w)
   ########Regression After Crisis#############
#regression
aop_w_40_60_beta1=[]
aop_w_40_60_beta1.append(optimal_w_after )
for i in range(len(weekday_after)):
    date = weekday_after.iloc[i][0] 
    securities = df_r.columns
    vols = []
    betas = []
    sigma_table = pd.DataFrame(columns=securities)

    for sec in securities:
        mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
        vols.append(mu_re)
        sigma_table[sec]= mu_sig
        betas.append(beta_i)
    sigma = sigma_table.cov()
    
#optimization
    op=get_weights(vols,sigma,aop_w_40_60_beta1[-1].flatten().tolist() ,betas,betaT=beta1,lamb=0.8)
    optimal_w =op [0]
    aop_w_40_60_beta1.append(optimal_w)

###########Regression for total Period##########
first_weekday=pd.DataFrame()
first_weekday=pd.concat([weekday_before,weekday_during,weekday_after])
first_weekday.index=range(len(first_weekday))
#regression
op_w_40_60_beta1=[]
op_w_40_60_beta1.append(optimal_w_before )
#op_r=[]
#op_r.append(optimal_r )
for i in range(len(first_weekday)):
    date = first_weekday.iloc[i][0] 
    securities = df_r.columns
    vols = []
    betas = []
    sigma_table = pd.DataFrame(columns=securities)

    for sec in securities:
        mu_re, mu_sig, beta_i = regress(40, 60, date, df_r[sec], FFM_df)
        vols.append(mu_re)
        sigma_table[sec]= mu_sig
        betas.append(beta_i)
    sigma = sigma_table.cov()
    
#optimization
    op=get_weights(vols,sigma,op_w_40_60_beta1[-1].flatten().tolist() ,betas,betaT=beta1,lamb=0.8)
    optimal_w =op [0]
    op_w_40_60_beta1.append(optimal_w)
return_before1 = optimal_portfolio_daily_return(bop_w_40_60_beta1,weekday_before) 
return_before2 = optimal_portfolio_daily_return(bop_w_40_60_beta1,weekday_before)
return_before3 = optimal_portfolio_daily_return(bop_w_90_120_beta1,weekday_before)

return_before4 = optimal_portfolio_daily_return(bop_w_40_60_beta2,weekday_before)
return_before5 = optimal_portfolio_daily_return(bop_w_40_90_beta2,weekday_before)
return_before6= optimal_portfolio_daily_return(bop_w_90_120_beta2,weekday_before)

return_before7= optimal_portfolio_daily_return(bop_w_40_60_beta3,weekday_before)
return_before8= optimal_portfolio_daily_return(bop_w_40_90_beta3,weekday_before)
return_before9= optimal_portfolio_daily_return(bop_w_90_120_beta3,weekday_before

return_during1 = optimal_portfolio_daily_return(dop_w_40_60_beta1,weekday_during)
return_during2 = optimal_portfolio_daily_return(dop_w_40_90_beta1,weekday_during)
return_during3 = optimal_portfolio_daily_return(dop_w_90_120_beta1,weekday_during)

return_during4 = optimal_portfolio_daily_return(dop_w_40_60_beta2,weekday_during)
return_during5 = optimal_portfolio_daily_return(dop_w_40_90_beta2,weekday_during)
return_during6 = optimal_portfolio_daily_return(dop_w_90_120_beta2,weekday_during)

return_during7 = optimal_portfolio_daily_return(dop_w_40_60_beta3,weekday_during)
return_during8 = optimal_portfolio_daily_return(dop_w_40_90_beta3,weekday_during)
return_during9 = optimal_portfolio_daily_return(dop_w_90_120_beta3,weekday_during)

after_return1 = optimal_portfolio_daily_return(aop_w_40_60_beta1,weekday_after)[0:1132]
after_return2 = optimal_portfolio_daily_return(aop_w_40_90_beta1,weekday_after)[0:1132]
after_return3 = optimal_portfolio_daily_return(aop_w_90_120_beta1,weekday_after)[0:1132]

after_return4 = optimal_portfolio_daily_return(aop_w_40_60_beta2,weekday_after)[0:1132]
after_return5 = optimal_portfolio_daily_return(aop_w_90_120_beta2,weekday_after)[0:1132]

after_return6 = optimal_portfolio_daily_return(aop_w_40_60_beta3,weekday_after)[0:1132]
after_return7 = optimal_portfolio_daily_return(aop_w_40_90_beta3,weekday_after)[0:1132]
after_return8 = optimal_portfolio_daily_return(aop_w_90_120_beta3,weekday_after)[0:1132]


return_full1=optimal_portfolio_daily_return(op_w_40_60_beta1,first_weekday)
return_full2=optimal_portfolio_daily_return(op_w_40_90_beta1,first_weekday)
return_full3 =optimal_portfolio_daily_return (op_w_90_120_beta1,first_weekday)

return_full4=optimal_portfolio_daily_return(op_w_40_60_beta2,first_weekday)
return_full5=optimal_portfolio_daily_return(op_w_40_90_beta2,first_weekday)
return_full6=optimal_portfolio_daily_return(op_w_90_120_beta2,first_weekday)

return_full7= optimal_portfolio_daily_return(op_w_40_60_beta3,first_weekday)
return_full8 = optimal_portfolio_daily_return(op_w_40_90_beta3,first_weekday)
return_full9 = optimal_portfolio_daily_return(op_w_90_120_beta3,first_weekday)

return_full1.tail()

rf = (df_mr/df_mr.shift(1)-1).dropna()
index=np.array(first_weekday).flatten().tolist()
rf.head()

df_mr = pd.read_csv('/content/gdrive/My Drive/Assignment Data/data/SPY.csv',index_col=['Date'],usecols=['Date','Adj Close'])
df_mr = pd.DataFrame((df_mr/df_mr.shift(1)-1).dropna())

#seperate sp500 date into three period 
spy_before = df_mr.iloc[df_mr.index.get_loc('2007-08-06'): df_mr.index.get_loc('2008-07-07')]
spy_dur = df_mr.iloc[df_mr.index.get_loc('2008-07-08'): df_mr.index.get_loc('2009-07-08')]
spy_after = df_mr.iloc[df_mr.index.get_loc('2009-07-06'): df_mr.index.get_loc('2013-12-')]
spy_tot= pd.concat([spy_before,spy_dur,spy_after])

import matplotlib.pyplot as plt

returnBeforeList = [return_before1,return_before2,return_before3,return_before4,return_before5,return_before6,return_before7,return_before8,return_before9]
labelListBefore = ["w_40_60_beta1","w_40_90_beta1","w_90_120_beta1","40_60_beta2","w_40_90_beta2","w_90_120_beta2","w_40_60_beta3","40_90_beta3","w_90_120_beta3"]

figDraw = plt.figure(figsize = (25,25))
j = 1
for i in returnBeforeList:
  dia = figDraw.add_subplot(3,3,j)
  dia.plot(spy_before["Adj Close"][0:228].cumsum(), c = "red", label = "SPY Before Crisis")
  dia.plot(i[0].cumsum(), c ="green" , label = labelListBefore[j-1]) 
  j = j + 1
  dia.legend()

import pandas as pd
import pandas_datareader as  web
import matplotlib.pyplot as pp
import datetime

returnDuringList = [return_during1,return_during2,return_during3,return_during4,return_during5,return_during6,return_during7,return_during8,return_during9]
labelListDuring = ["w_40_60_beta1","w_40_90_beta1","w_90_120_beta1","w_40_60_beta2","w_40_90_beta2","w_90_120_beta2","w_40_60_beta3","w_40_90_beta3","w_90_120_beta3"]

figDraw = plt.figure(figsize = (25,25))
j = 1
for i in returnDuringList:
  dia = figDraw.add_subplot(3,3,j)
  dia.plot(spy_dur["Adj Close"][0:228].cumsum(), c = "red", label = "SPY During Crisis")
  dia.plot(i[0].cumsum(), c ="green" , label = labelListDuring[j-1]) 
  j = j + 1
  dia.legend()

returnAfterList = [after_return1,after_return2,after_return3,after_return4,after_return5,after_return6,after_return7,after_return8]
labelListAfter = ["40_60_beta1","w_40_90_beta1","w_90_120_beta1","40_60_beta2","90_120_beta2","40_60_beta3","40_90_beta3","aop_90_120_beta3"]

figDraw = plt.figure(figsize = (30,30))
j = 1
for i in returnAfterList:
  dia = figDraw.add_subplot(3,3,j)
  dia.plot(spy_after["Adj Close"].cumsum(), c = "red", label = "SPY After Crisis")
  dia.plot(i[0].cumsum(), c ="green" , label = labelListAfter[j-1]) 
  j = j + 1
  dia.legend()

import math

finalBefore = {}
j = 0

for i in returnBeforeList:
  a = [i[0].cumsum().mean(),i[0].cumsum().std() * math.sqrt(252),i[0].cumsum().skew(),i[0].cumsum().kurt(),i[0].cumsum().mean() / (i[0].cumsum().std() * math.sqrt(252)),i[0].cumsum().var(),min(i[0].cumsum()),max(i[0].cumsum())]
  finalBefore[labelListBefore[j]] = a
  j = j + 1

finalDuring = {}
j = 0

for i in returnDuringList:
  a = [i[0].cumsum().mean(),i[0].cumsum().std() * math.sqrt(252),i[0].cumsum().skew(),i[0].cumsum().kurt(),i[0].mean() / i[0].std() * math.sqrt(252),i[0].cumsum().var(),min(i[0].cumsum()),max(i[0].cumsum())]
  finalDuring[labelListDuring[j]] = a
  j = j + 1

finalAfter = {}
j = 0

for i in returnAfterList:
  a = [i[0].cumsum().mean(),i[0].cumsum().std() * math.sqrt(252),i[0].cumsum().skew(),i[0].cumsum().kurt(),i[0].cumsum().mean() / (i[0].cumsum().std() * math.sqrt(252)),i[0].cumsum().var(),min(i[0].cumsum()),max(i[0].cumsum())]
  finalAfter[labelListAfter[j]] = a
  j = j + 1



indexes = ["Mean","Volatility","Skewness","Kurtosis","Sharp Ratio","Variance","Minimum Return","Maximum Return"]

dfBefore = pd.DataFrame(data=finalBefore, index = indexes)
dfDuring = pd.DataFrame(data=finalDuring, index = indexes)
dfAfter = pd.DataFrame(data=finalAfter, index = indexes)

dfBefore
dfDuring
dfAfter
j = 0
for i in returnBeforeList:
  count, bins, ignored = plt.hist(i[0].cumsum(), bins = 20, density=True, stacked=True, color = 'r', label = labelListBefore[j-1])
  j = j + 1
  plt.legend()
  plt.show()
j = 0
for i in returnDuringList:
  count, bins, ignored = plt.hist(i[0].cumsum(), bins = 20, density=True, stacked=True, color = 'blue', label = labelListDuring[j-1])
  j = j + 1
  plt.legend()
  plt.show()

j = 0
for i in returnAfterList:
  count, bins, ignored = plt.hist(i[0].cumsum(), bins = 20, density=True, stacked=True, color = 'green', label = labelListAfter[j-1])
  j = j + 1
  plt.legend()
  plt.show()
returnTotalList = [return_full1,return_full2,return_full3,return_full4,return_full5,return_full6,return_full7,return_full8,return_full9]
labelListTotal = ["w_40_60_beta1","w_40_90_beta1","w_90_120_beta1","w_40_60_beta2","w_40_90_beta2","w_90_120_beta2","w_40_60_beta3","w_40_90_beta3","w_90_120_beta3"]

figDraw = plt.figure(figsize = (25,25))
j = 1
for i in returnTotalList:
  dia = figDraw.add_subplot(3,3,j)
  dia.plot(spy_tot["Adj Close"].cumsum(), c = "blue", label = "SPY")
  dia.plot(i[0].cumsum(), c ="green" , label = labelListTotal[j-1]) 
  j = j + 1
  dia.legend()
